In [ ]:
!pip install requests elasticsearch tqdm tiktoken

In [37]:
import requests 
import json
import os

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [26]:
def elasticsearch_search(query, documents, filter_dict, size_answers):
    from elasticsearch import Elasticsearch
    es_client = Elasticsearch("http://localhost:9200")

    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }

    index_name = "course-homework-01"

    try:
        es_client.indices.create(index=index_name, body=index_settings)
        from tqdm.auto import tqdm
        for doc in tqdm(documents):
            es_client.index(index=index_name, document=doc)
    except:
        print("The index is created already")
        

    search_query = {
        "size": size_answers,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    search_query["query"]["bool"].update(filter_dict)

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit)

    return result_docs

In [27]:
query = "How do I execute a command in a running docker container?"
filter_dict = {}
results = elasticsearch_search(query, documents, filter_dict, 1)
print([elem["_score"] for elem in results])

The index is created already
[84.050095]


In [28]:
query = "How do I execute a command in a running docker container?"
filter_dict = {
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
results = elasticsearch_search(query, documents, filter_dict, 3)
print([elem["_source"]["question"] for elem in results])

The index is created already
['How do I debug a docker container?', 'How do I copy files from my local machine to docker container?', 'How do I copy files from a different folder into docker container’s working directory?']


In [29]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question =  {doc["question"]}, text = {doc["text"]}) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [31]:
query = "How do I execute a command in a running docker container?"
filter_dict = {
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
results = elasticsearch_search(query, documents, filter_dict, 3)
search_results = [elem["_source"] for elem in results]
prompt = build_prompt(query, search_results)
print(len(prompt))



The index is created already
1508


In [36]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
print(len(encoding.encode(prompt)))

354


In [42]:
def llm(prompt):

    try:
        OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]
    except KeyError:
        with open("../openrouter_api_key.txt", "r") as f:
            OPENROUTER_API_KEY = f.read()
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}"
        },
        data=json.dumps({
            "model": "google/gemma-7b-it:free",
            "messages": [
            { "role": "user", "content": prompt }
            ]
        })
        )
    
    return response.json()["choices"][0]["message"]["content"].strip()

In [43]:
answer = llm(prompt)
print(answer)

To execute a command in a running docker container, use the `docker exec -it <container-id> bash` command.
